In [3]:
import json
from time import time
import pandas as pd
from pathlib import Path
from dpmm.pipelines import PrivBayesPipeline

/tmp/ipykernel_9521/3077299844.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data

In [4]:
wine_dir = Path().parent / "wine"

df = pd.read_pickle(wine_dir / "wine.pkl.gz")
with (wine_dir / "wine_bounds.json").open("r") as f:
    domain = json.load(f)

df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4192,white,6.3,0.21,0.29,11.7,0.048,49.0,147.0,0.99482,3.22,0.38,10.8,0
2157,white,5.9,0.21,0.24,12.1,0.044,53.0,165.0,0.99690,3.25,0.39,9.5,0
631,red,10.4,0.28,0.54,2.7,0.105,5.0,19.0,0.99880,3.25,0.63,9.5,0
3410,white,7.6,0.38,0.20,3.4,0.046,9.0,116.0,0.99440,3.15,0.41,9.4,0
3117,white,8.4,0.23,0.49,7.8,0.035,22.0,95.0,0.99350,3.04,0.34,12.0,1


# Build pipeline

In [5]:
pipeline = PrivBayesPipeline(
    epsilon=1,  # Privacy budget for generate model
    proc_epsilon=None,  # Privacy budget for data processing 
    delta=1e-5,  # Delta Setting in (eps,  delta) differential privacy
    max_model_size=1.0, # Maximum Model Size
    n_bins=20,
    gen_kwargs={
        "n_iters": 100,
        "degree": 2,
    },
    n_jobs=1

)

# Fit pipeline

In [6]:
print("Fitting PrivBayes pipeline...", end='\r')
start_time = time()
# The number of rows to generate
pipeline.fit(df, domain)
fit_time = time()

print(f"Fitting PrivBayes pipeline - Took {fit_time - start_time:.2f} seconds - Memory Usage : {pipeline.model_size:.2f} Mb")

Fitting PrivBayes pipeline - Took 11.19 seconds - Memory Usage : 0.11 Mb


# Generate step
Deterministic with random state set to 2025

In [7]:
print("Generating synthetic data ...", end="\r")
# Generate synthetic data
synth_df = pipeline.generate(df.shape[0])
gen_time = time()

print(f"Generating synthetic data - Took {gen_time - fit_time:.2f} seconds")

display(synth_df.head())

Generating synthetic data - Took 4.77 seconds


,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,6.760604,0.257941,0.425834,5.366838,0.041819,34.124896,135.907825,0.994331,3.254318,0.409451,13.279321,1
1,white,7.513120,0.328950,0.314771,0.735613,0.289869,29.814339,162.516000,0.990865,3.125501,0.656055,10.739345,1
2,white,6.841315,1.007450,0.384935,10.849934,0.417861,48.576109,346.265121,0.999881,3.188643,0.632659,14.280409,1
3,red,5.530003,0.436880,0.191151,2.216846,0.402229,12.705759,105.279513,0.996131,3.347789,0.659616,9.782915,0
4,red,5.279018,0.437999,1.401743,24.479508,0.052499,10.645054,7.525711,0.999642,3.414530,0.666445,8.402302,1
